Don't write if schema doesn't match 

Schema Enforcement : Delta Lake or Delta Tables uses Schema Validation on "writes". If any new column add at source and which is not their at Target file generally throws error. But if you are saving file as delta format , delta format uses Schema Enforcement on Write, even column are differe from src to target . But it will merge 

Rules : 
1.If incoming file has more number of columns compare to Target then we can't append 
2.If incoming file has less number of column compare to Target then we can append 



In [0]:
dbutils.fs.ls('mnt/DeltaLake/Test/schemaevol/')

[FileInfo(path='dbfs:/mnt/DeltaLake/Test/schemaevol/SchemaLessCols.csv', name='SchemaLessCols.csv', size=83257, modificationTime=1709031342000),
 FileInfo(path='dbfs:/mnt/DeltaLake/Test/schemaevol/SchemaMoreCols.csv', name='SchemaMoreCols.csv', size=111809, modificationTime=1709031341000)]

In [0]:
path_schema_less = 'dbfs:/mnt/DeltaLake/Test/schemaevol/SchemaLessCols.csv'
path_schema_more = 'dbfs:/mnt/DeltaLake/Test/schemaevol/SchemaMoreCols.csv'

In [0]:
#read csv file and write to delta 

df = spark.read.format('csv').option('Header', True).option('inferSchema', True).load(path_schema_more)

In [0]:
df.printSchema()

root
 |-- Education_Level: string (nullable = true)
 |-- Line_Number: integer (nullable = true)
 |-- Employed: integer (nullable = true)
 |-- Unemployed: integer (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Date_Inserted: string (nullable = true)
 |-- dense_rank: integer (nullable = true)
 |-- Max_Salary_USD: integer (nullable = true)



In [0]:
#write this df as Delta Table 

#error : A schema mismatch detected when writing to the Delta table (Table ID: 2f34627c-132d-4c78-adc7-823172082f02).

df.write.format('delta').mode('append').saveAsTable('`delta`.DeltaSpark')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3981348911901521>, line 5
      1 #write this df as Delta Table 
      2 
      3 #error : A schema mismatch detected when writing to the Delta table (Table ID: 2f34627c-132d-4c78-adc7-823172082f02).
----> 5 df.write.format('delta').mode('append').saveAsTable('`delta`.DeltaSpark')

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1586, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1584 if format is not None:
   1

In [0]:
df.write.format('delta').mode('append').option('mergeSchema', 'True').saveAsTable('`delta`.DeltaSpark')

In [0]:
%sql

select * from `delta`.deltaspark

Education_Level,Line_Number,Employed,Unemployed,Industry,Gender,Date_Inserted,dense_rank,Max_Salary_USD
Bachelor's Degree,326,118000,2000,Agriculture,Male,6/1/1999,1,120000
High School Diploma,195,11134000,240000,Agriculture,Female,9/1/1996,2,70000
High School Diploma,129,675000,15000,Retail,Female,5/1/1995,3,70000
Master's Degree,406,131500,3500,Agriculture,Male,2/1/2001,4,150000
High School,1463,4031000,115000,Retail Trade,Male,12/1/2000,5,70000
Bachelor's Degree,426,136900,4100,Manufacturing,Male,7/1/2001,6,120000
Bachelor's Degree,458,136900,4100,Agriculture,Male,3/1/2002,6,120000
Bachelor's Degree,410,136900,4100,Agriculture,Male,3/1/2001,6,120000
High School,1462,4018000,123000,Retail Trade,Female,11/1/2000,7,70000
Associate's Degree,507,53350,1650,Education,Male,3/1/2003,8,90000


In [0]:
#lets check the column count 

df_existance = spark.read.format('delta').load('dbfs:/user/hive/warehouse/delta.db/deltaspark')

column_count = len(df_existance.columns)
print(f'existing: {column_count}')
column_count_new = len(df.columns)
print(f'new: {column_count_new}')

#find out which column is missing  

df1 = set(df_existance.columns)
dfN = set(df.columns) 

missing = dfN - df1
print(missing) #{'Max_Salary_USD'} is missing in df_exisitance 

existing: 8
new: 9
{'Max_Salary_USD'}


In [0]:
#read less number of columns 

df_less = spark.read.format('csv').option('Header', True).option('inferSchema', True).load(path_schema_less)

In [0]:
df_less.columns

['Education_Level',
 'Line_Number',
 'Employed',
 'Unemployed',
 'Industry',
 'Gender',
 '_c6']

In [0]:
dfL = set(df_less.columns)

addlcolumns = df1 - dfL
print(addlcolumns)

{'Date_Inserted', 'dense_rank'}


In [0]:
## for datatype missmatch use option('overwriteSchema', 'True')